In [1]:
import os
import json
import glob
import shutil
from math import dist
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.font_manager import FontProperties

In [2]:
degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg', '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']
acupoints_name = ["HX6(肛門)", "HX3(尿道)", "HX1(耳中)", "TF4(神門)", "AH9(腰柢椎)", "AH11(胸椎)", "CO5(十二指腸)", 
                  "CO4(胃)", "CO3(賁門)", "CO14(肺)", "CO15(心)", "CO18(內分泌)", "HX2(輪2)", "AT2(緣中)", 
                  "LO8(扁桃體)", "LO1(牙)" , "LO2(舌)", "LO3(頷)", "LO4(垂前)", "LO5(眼)", "LO6(內耳)"]
ear_types = ["free","attached"]
cm = {"cather":4.72, "daniel":4.74, "dominic":3.9, "jack":3.91, "jakaria":4.20, "jimmy":4.60, "vicky":4.69, "wayne":4.75,
      "gaby":4.05, "matt":5.0, "toby":4.26, "wendy":4.14, "willy":4.51}

In [3]:
class localization_error():
    def __init__(self, src):
        self.src = src
        
    def read_json(self, file):      
        with open(file, "r") as f:
            gt_file = json.load(f)
        return gt_file["annotations"]
    
    def gt_df(self, gt_keypoints):
        df = pd.DataFrame()
        for i in range(0, len(gt_keypoints)):
            kpts_pd = {}
            for j in range(0, int(gt_keypoints[i]["num_keypoints"])):
                kpts_pd[j] = str(gt_keypoints[i]["keypoints"][3*j])+","+str(gt_keypoints[i]["keypoints"][3*j+1])+","+str(gt_keypoints[i]["keypoints"][3*j+2])
            ser = pd.DataFrame(data=kpts_pd, index = [i])
            df = pd.concat([df, ser])
        return df
    
    def read_csv(self, fpath1):
        error_df = pd.read_csv(fpath1, index_col = 0)
        return error_df
    def generate_gt_csv(self, src, ear_types, degrees):   #generate gt.csv
        for ear_type in ear_types:
            names = os.listdir(os.path.join("..", src, ear_type, "result"))
            for name in names:
    
                file = os.path.join("..", src, ear_type, "result", name, "test_json", "test.json")
                gt_keypoints = self.read_json(file)
                    
                df = self.gt_df(gt_keypoints)
                count = 0
                for deg in degrees:
                    
                    frame_count = len(self.read_csv(os.path.join("..", src, ear_type, "result", name, "pred", deg, "pred.csv")))
                    if not os.path.isdir(os.path.join("..", src, ear_type, "result", name, "ground_truth",deg)):
                        os.makedirs(os.path.join("..", src, ear_type, "result", name, "ground_truth",deg))
                    df.iloc[count:count+frame_count,:].to_csv(os.path.join("..", src, ear_type, "result", name, "ground_truth", deg, "gt.csv"))
                    count = count + frame_count
                    
    def convert_to_error_csv(self, src, ear_types, degrees, cm):
        self.generate_gt_csv(src, ear_types, degrees)
        for ear_type in ear_types:
            names = os.listdir(os.path.join("..", src, ear_type, "result"))
            for name in names:
                for deg in degrees:
                    gt = os.path.join("..", src, ear_type, "result", name, "ground_truth", deg, "gt.csv")
                    pred = os.path.join("..", src, ear_type, "result", name, "pred", deg, "pred.csv")

                    gt_df = self.read_csv(gt)
                    pred_df = self.read_csv(pred)

                    error_df = pd.DataFrame(columns=gt_df.columns, index = gt_df.index)

                    assert len(gt_df) == len(pred_df), "different frames"
                    for i in range(0, len(gt_df)):

                        scale0_x = float(gt_df.iloc[i,0].split(",")[0])
                        scale0_y = float(gt_df.iloc[i,0].split(",")[1])

                        scale11_x = float(gt_df.iloc[i,11].split(",")[0])
                        scale11_y = float(gt_df.iloc[i,11].split(",")[1])

                        scale = cm[name] / dist((scale0_x, scale0_y), (scale11_x, scale11_y))

                        for j in range(0, len(gt_df.columns)):
                            if gt_df.iloc[i,j].split(",")[2] == "2":
                                pred_x = float(pred_df.iloc[i,j].split(",")[0])
                                pred_y = float(pred_df.iloc[i,j].split(",")[1])

                                gt_x = float(gt_df.iloc[i,j].split(",")[0])
                                gt_y = float(gt_df.iloc[i,j].split(",")[1])

                                error_df.iloc[i,j] = scale * dist((gt_x, gt_y), (pred_x, pred_y))



                    error_path = os.path.join("..", src, ear_type, "result", name, "error",deg)
                    if not os.path.isdir(error_path):
                        os.makedirs(error_path)
                    error_df.to_csv(os.path.join(error_path, "error.csv"))

In [4]:
class localization_error_Draw():
    def __init__(self, src):
        self.src = src